In [1]:
import os
import numpy as np
import pickle
import pandas as pd
from scipy.stats import zscore
from brainbox.io.one import SessionLoader
from sklearn.preprocessing import StandardScaler
import gc
import concurrent.futures

from functions import get_left_licks, resample_common_time, interpolate_nans
from one.api import ONE
one = ONE(mode='remote')


In [2]:
prefix = '/home/ines/repositories/'
prefix = '/Users/ineslaranjeira/Documents/Repositories/'

# Get Kcenia's sessions

In [3]:
kcenia_sessions = ['d7bf2611-00bc-42d3-95c6-ac120753cd94',
                   '72a5e765-efaf-49a5-831b-3cc35ec76517',
                    '933765b5-1f4e-4877-b889-7cc87f6ddb64',
                    'a196fce9-7eae-4c11-ae01-25a9aa59b593',
                    'd7bf2611-00bc-42d3-95c6-ac120753cd94',
                    'e69221c7-b533-42dc-8631-000279a45a70',
                    '364d72f6-483c-4146-88df-e77e86f22a60',
                    '89a9b132-7cda-4d97-8d28-448119c0910e',
                    'a3657685-f782-41e6-bc94-2fb1061b1fec',
                    'acdb349a-0181-4455-82c1-2cbdc24ca362',
                    'b27c0221-781d-4519-bd14-9a03938edc12',
                    'e505245e-585c-451e-b590-60f3ef714a89',
                    'f9b1dafa-1ec0-46e2-a72c-987535b72bb6',
                    '04482c20-f984-4d90-97d6-6791bae1e6f7',
                    '291e25f7-b35f-4d11-815f-1429127059d4',
                    '5853e474-2c8f-448f-8ef8-c753eaceb408',
                    '604dc7eb-533f-4d32-a1a3-064f7144656e',
                    '69544b1b-7788-4b41-8cad-2d56d5958526',
                    '89b8ef70-e620-49c2-a0f7-09890ba9fc0e',
                    '941df82c-9894-4280-bf3c-09323658b299',
                    '9ea8fa32-3cbf-4ea1-a341-285837caa137',
                    'bd5dbb12-dbfd-488c-b5d9-2b06bed078dd',
                    'd78d8da7-980d-4550-a039-8217abe51d9a',
                    'f5369fa4-4bc2-4c65-9217-bea27df0f9ca',
                    '3bef683f-46a4-4b98-8b53-8eb0d5bdf82b',
                    '475d6c49-1e0f-49db-b591-645c3283926e',
                    '8c2639e9-ad67-4c07-bdc0-4a5ba994375c',
                    'a06c142a-9b90-4639-a66e-0ed464eeb476',
                    'd3b1eae9-7b19-4d09-9560-23092cce572c',
                    'd8b48595-6aa5-4e4b-b457-13b1b936248c',
                    'f0cfd8b5-4568-4944-96c9-23d75aed659c',
                    'f2343157-4824-4fa2-84c1-a8bfebe21a8f',
                    'fa9efd38-29f9-41a6-a005-21c9fc4be5dc',
                    '404d1b7e-f6f2-4122-bd2d-9caa6a0e7777',
                    '0de78321-9dce-4964-af4b-d0f00a41c647',
                    '929662b6-4182-4ee7-8a50-ec788520c9e2',
                    '68f894bf-2268-4773-aa8d-ac8ab168320e',
                    'dd549116-68a1-462d-90ef-f4e0ec5c69fc',
                    '6f224adf-e791-4255-b6ab-e4e1f059813a',
                    'e470c52d-25bc-4dac-9679-2dc0b7a3677f',
                    '1f959c77-862c-4da1-a454-74a6e99ff33e',
                    '12f1d635-8a2a-49fe-ad4d-efa33b89301b',
                    '163fa79b-9b33-43ee-8d58-335f4c12d370']

data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/kcenia/'

# Get BWM sessions

In [4]:
""" Load BWM data post-QC """

data_path = prefix + '/representation_learning_variability/paper-individuality/'
filename = '1_bwm_qc_07-10-2025'

data_path = prefix + 'representation_learning_variability/Video and wheel/Video QC/'
filename = 'final_lp_qc07-10-2025'


bwm_query = pickle.load(open(data_path+filename, "rb"))

In [ ]:
# Loop through animals
sessions = bwm_query['eid'].unique()
data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/1_camera_setup/'

sessions = kcenia_sessions
data_path = prefix + 'representation_learning_variability/paper-individuality/data/design_matrices/kcenia/'

os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)

    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))

len(sessions_to_process)

10

In [11]:
def process_design_matrix(session):

    file_path = one.eid2path(session)
    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    """ LOAD VARIABLES """
    sl = SessionLoader(eid=session, one=one)
    sl.load_pose(views=['left'], tracker='lightningPose')
    sl.load_session_data(trials=True, wheel=True, motion_energy=True)

    # Check if all data is available
    if np.sum(sl.data_info['is_loaded']) >= 4:

        # Poses
        poses = sl.pose
        lc_t = np.asarray(poses['leftCamera']['times'])
        # Motion energy
        me = sl.motion_energy
        mel_t = lc_t
        motion_energy_l = interpolate_nans(me['leftCamera']['whiskerMotionEnergy'], 'left')

        # Licks
        features = ['tongue_end_l_x', 'tongue_end_l_y', 'tongue_end_r_x', 'tongue_end_r_y']
        lick_t, licks = get_left_licks(poses, features, common_fs=60)

        # Paws
        l_paw_x = interpolate_nans(poses['leftCamera']['paw_r_x'], 'left')
        l_paw_y = interpolate_nans(poses['leftCamera']['paw_r_y'], 'left')
        r_paw_x = interpolate_nans(poses['leftCamera']['paw_l_x'], 'left')
        r_paw_y = interpolate_nans(poses['leftCamera']['paw_l_y'], 'left')
        l_paw_t = lc_t
        r_paw_t = lc_t
        # Wheel
        wheel = sl.wheel
        wheel_t = np.asarray(wheel['times'], dtype=np.float64)
        wheel_vel = wheel['velocity'].astype(np.float32)

        # Common resampling window
        onset = max(lc_t.min(), wheel_t.min(), lick_t.min())
        offset = min(lc_t.max(), wheel_t.max(), lick_t.max())
        fs = 30
        ref_t = np.arange(onset, offset, 1 / fs, dtype=np.float64)

        # Restrict to time window
        def restrict(t, x):
            mask = (t >= onset) & (t <= offset)
            return t[mask], x[mask]

        mel_t, motion_energy_l = restrict(mel_t, motion_energy_l)
        wheel_t, wheel_vel = restrict(wheel_t, wheel_vel)
        l_paw_t_x, l_paw_x = restrict(l_paw_t, l_paw_x)
        l_paw_t_y, l_paw_y = restrict(l_paw_t, l_paw_y)
        r_paw_t_x, r_paw_x = restrict(r_paw_t, r_paw_x)
        r_paw_t_y, r_paw_y = restrict(r_paw_t, r_paw_y)
        lick_t, licks = restrict(lick_t, licks)

        # Resample
        mel_down, rt = resample_common_time(ref_t, mel_t, motion_energy_l, kind='linear')
        wh_down, _ = resample_common_time(ref_t, wheel_t, wheel_vel, kind='linear')
        lk_down, _ = resample_common_time(ref_t, lick_t, licks, kind='nearest')
        lpx_down, _ = resample_common_time(ref_t, l_paw_t_x, l_paw_x, kind='linear')
        lpy_down, _ = resample_common_time(ref_t, l_paw_t_y, l_paw_y, kind='linear')
        rpx_down, _ = resample_common_time(ref_t, r_paw_t_x, r_paw_x, kind='linear')
        rpy_down, _ = resample_common_time(ref_t, r_paw_t_y, r_paw_y, kind='linear')

        # Create design matrix
        design_matrix = pd.DataFrame({
            'Bin': rt,
            'Lick count': lk_down.astype(np.int8),
            'avg_wheel_vel': wh_down,
            'whisker_me': mel_down,  # zscore(mel_down, nan_policy='omit'),
            'l_paw_x': lpx_down,
            'l_paw_y': lpy_down,
            'r_paw_x': rpx_down,
            'r_paw_y': rpy_down,
        })

        # """ LOAD TRIAL DATA """
        session_trials = sl.trials
        session_start = list(session_trials['goCueTrigger_times'])[0]
        design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]

        """ SAVE DATA """       
        # Save unnormalized design matrix
        filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
        design_matrix.to_parquet(filename, compression='gzip')  

        # Save trials
        filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
        session_trials.to_parquet(filename, compression='gzip')  
        
        del design_matrix, session_trials, sl
        gc.collect()

    else:
        print('Data missing for session '+session)  


def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

In [14]:
for s, session in enumerate(sessions_to_process):
    try:
        process_design_matrix(session)
    except:
        print(session)

2026-02-08 14:44:42 INFO     one.py:1288 Loading trials data


HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_trials.table.810129d2-4bc1-4e17-b780-a4ad1445d894.pqt
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_trials.stimOffTrigger_times.24f0e7a3-2d5c-44c1-9587-658366d00fa6.npy
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_trials.stimOff_times.a0626e28-a0b6-476a-90cd-6baa69e15891.npy
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_trials.goCueTrigger_times.9c916d90-d45f-40df-8e1e-7edfe5f1d06c.npy
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_trials.included.b77a8e8d-4cbf-4ce8-b394-524dca202e6f.npy
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects

2026-02-08 14:44:43 WARNING  one.py:1292 Could not load trials data.
2026-02-08 14:44:43 INFO     one.py:1288 Loading wheel data


HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_wheel.position.6ceaa674-3a5b-4555-8103-cade741448fd.npy
HTTP Error 308: Permanent Redirect http://ibl.flatironinstitute.org/mainenlab/Subjects/ZFM-05235/2023-08-24/001/alf/_ibl_wheel.timestamps.edbb648c-3e45-43ba-bc11-2016eb826d03.npy


2026-02-08 14:44:44 WARNING  one.py:1292 Could not load wheel data.
2026-02-08 14:44:44 INFO     one.py:1288 Loading motion_energy data
2026-02-08 14:44:44 WARNING  one.py:1292 Could not load motion_energy data.
2026-02-08 14:44:44 INFO     one.py:1288 Loading pupil data
Data missing for session 933765b5-1f4e-4877-b889-7cc87f6ddb64
2026-02-08 14:44:45 INFO     one.py:1288 Loading trials data


...


(S3) /Users/ineslaranjeira/Downloads/FlatIron/mainenlab/Subjects/ZFM-04019/2022-09-20/001/alf/_ibl_wheel.timestamps.npy: 100%|██████████| 1.53M/1.53M [00:05<00:00, 258kB/s]
(S3) /Users/ineslaranjeira/Downloads/FlatIron/mainenlab/Subjects/ZFM-04019/2022-09-20/001/alf/_ibl_wheel.position.npy: 100%|██████████| 1.53M/1.53M [00:03<00:00, 485kB/s]


2026-02-08 14:48:28 INFO     one.py:1288 Loading motion_energy data


(S3) /Users/ineslaranjeira/Downloads/FlatIron/mainenlab/Subjects/ZFM-04019/2022-09-20/001/alf/leftCamera.ROIMotionEnergy.npy: 100%|██████████| 919k/919k [00:01<00:00, 781kB/s]

2026-02-08 14:48:30 WARNING  one.py:1292 Could not load motion_energy data.
2026-02-08 14:48:30 INFO     one.py:1288 Loading pupil data



(S3) /Users/ineslaranjeira/Downloads/FlatIron/mainenlab/Subjects/ZFM-04019/2022-09-20/001/alf/_ibl_leftCamera.features.pqt: 100%|██████████| 2.32M/2.32M [00:01<00:00, 1.78MB/s]
